# 🚀 ARB-BOT — INGESTA MEJORADA V2.0

**Cambios principales:**
- ✅ Usa `pdfplumber` (mejor que PyPDF2)
- ✅ Función `fix_broken_words()` que repara palabras cortadas
- ✅ Limpieza automática en todo el proceso

**Antes de ejecutar:**
1. Sube el PDF: `MANUAL DE CONVIVENCIA ESCOLAR ROLDANISTA 2023.pdf`
2. Ejecuta celda por celda


In [ ]:
# 1) Instalar dependencias
!pip install -q supabase sentence-transformers psycopg2-binary python-docx
print('✅ Dependencias instaladas')


In [ ]:
# 2) Imports
import os
import json
import re
import psycopg2
from psycopg2.extras import execute_values
from sentence_transformers import SentenceTransformer
from docx import Document  # Para leer archivos Word
from datetime import datetime, timezone
from pathlib import Path

print('✅ Imports listos')


In [ ]:
# 3) Configuración - VERIFICA TUS CREDENCIALES DE SUPABASE
# ============================================
# Estas son las credenciales de conexión DIRECTA a PostgreSQL de Supabase
# (No usamos el cliente de Supabase, sino conexión directa a la BD)
# ============================================

DB_HOST = "aws-1-us-east-1.pooler.supabase.com"
DB_NAME = "postgres"
DB_USER = "postgres.ympekltzqzlsbdgbzbpz"
DB_PASS = "Z32pp23z$$1124$$"
DB_PORT = "6543"

SCHEMA = "vecs"
TABLE = "arbot_documents"

# ⚠️ CAMBIA EL NOMBRE DEL ARCHIVO SI ES DIFERENTE
WORD_FILE = "MANUAL DE CONVIVENCIA ESCOLAR ROLDANISTA 2023.docx"

def get_connection():
    """Conecta directamente a PostgreSQL de Supabase usando psycopg2."""
    return psycopg2.connect(
        host=DB_HOST, 
        dbname=DB_NAME, 
        user=DB_USER, 
        password=DB_PASS, 
        port=DB_PORT
    )

# Test conexión
print('🔗 Probando conexión a Supabase PostgreSQL...')
try:
    conn_test = get_connection()
    conn_test.close()
    print('✅ Conexión a Supabase OK')
except Exception as e:
    print(f'❌ Error de conexión: {e}')
    print('   Verifica que las credenciales sean correctas.')


In [ ]:
# 4) 🔧 FUNCIÓN MEJORADA: Limpiar palabras cortadas (sin ser agresiva)
def fix_broken_words(text: str) -> str:
    """
    Repara palabras cortadas por mal OCR/extracción.
    SOLO junta cuando hay UNA sola letra seguida de espacio y otra letra.
    Ejemplos: 'institu ción' → 'institución'
    NO debe juntar palabras completas.
    """
    if not text:
        return text
    
    # Patrón CONSERVADOR: solo junta cuando hay 1-2 letras sueltas
    # Ejemplo: "a ción" → "ación", "Rol dán" → "Roldán"
    
    # Caso 1: Una letra minúscula suelta + espacio + palabra
    # "a ción" → "ación"
    text = re.sub(r'\b([a-záéíóúñü]{1,2}) ([a-záéíóúñü])', r'\1\2', text)
    
    # Caso 2: Palabra + espacio + una letra minúscula suelta
    # "instituc ión" → "institución"  
    text = re.sub(r'([a-záéíóúñü]) ([a-záéíóúñü]{1,3})\b', r'\1\2', text)
    
    # Caso 3: Mayúscula + espacio + minúscula (inicio de palabra cortada)
    # "Rol dán" → "Roldán"
    text = re.sub(r'([A-ZÁÉÍÓÚÑÜ][a-záéíóúñü]*) ([a-záéíóúñü]{1,3})\b', r'\1\2', text)
    
    # NO tocar mayúsculas completas para no pegar títulos
    
    # Limpiar espacios múltiples
    text = re.sub(r'  +', ' ', text)
    
    return text.strip()

# Test de la función
print("🧪 Test de limpieza (versión conservadora):")
tests = [
    ("institu ción", "institución"),
    ("ESTUDIANTE S", "ESTUDIANTE S"),  # NO debe cambiar mayúsculas
    ("Rol dán", "Roldán"),
    ("derech o al debido proces o", "derecho al debido proceso"),
    ("MANUAL DE CONVIVENCIA", "MANUAL DE CONVIVENCIA"),  # NO debe cambiar
]
for original, esperado in tests:
    resultado = fix_broken_words(original)
    status = "✅" if resultado == esperado else "❌"
    print(f"  {status} '{original}' → '{resultado}' (esperado: '{esperado}')")

print('\n✅ Función de limpieza lista')


In [ ]:
# 5) Función para leer archivo Word (.docx)
# ¡Mucho más limpio que PDF!

def read_word_extract_text(docx_path: str):
    """
    Extrae texto de un archivo Word (.docx).
    El texto viene LIMPIO, sin problemas de OCR ni palabras cortadas.
    """
    doc = Document(docx_path)
    paragraphs = []
    full_text = ""
    
    print(f"📄 Leyendo documento Word: {docx_path}")
    
    for i, para in enumerate(doc.paragraphs):
        text = para.text.strip()
        if text:  # Solo párrafos con contenido
            paragraphs.append({"index": i, "text": text})
            full_text += text + "\n\n"
    
    print(f'✅ Documento leído: {len(paragraphs)} párrafos con texto')
    
    # Mostrar preview
    print("\n📋 Preview (primeros 500 caracteres):")
    print("-"*40)
    print(full_text[:500])
    print("-"*40)
    
    return {"text": full_text, "paragraphs": paragraphs}

print('✅ Función de extracción de Word lista')


In [ ]:
# 6) Utilidades
def clean_text(s):
    """Limpia espacios extra."""
    if not s: return s
    return re.sub(r'\s+', ' ', s).strip()

print('✅ Utilidades listas')


In [ ]:
# 7) Utilidades y Chunking jerárquico
def clean_text(s):
    if not s: return s
    s = fix_broken_words(s)
    return re.sub(r'\s+', ' ', s).strip()

def chunk_hierarchical_legal(full_text, max_tokens=1600):
    lines = full_text.split('\n')
    chunks = []
    current = {'title': None, 'chapter': None, 'article': None, 'text_lines': []}

    title_pat = re.compile(r'(?i)^(TÍTULO|TITULO)\b')
    chapter_pat = re.compile(r'(?i)^(CAPÍTULO|CAPITULO)\b')
    article_pat = re.compile(r'(?i)^(ARTÍCULO|ARTICULO|Artículo|Articulo)\s*\d+')

    def save():
        txt = '\n'.join(current['text_lines']).strip()
        if txt:
            cleaned = clean_text(txt)
            chunks.append({'text': cleaned, 'meta': {k: current.get(k) for k in ('title','chapter','article')}})

    for line in lines:
        s = line.strip()
        if not s:
            current['text_lines'].append('')
            continue
        if title_pat.match(s):
            save()
            current = {'title': s, 'chapter': None, 'article': None, 'text_lines': [s]}
            continue
        if chapter_pat.match(s):
            save()
            current['chapter'] = s
            current['text_lines'] = [s]
            continue
        if article_pat.match(s):
            save()
            current['article'] = s
            current['text_lines'] = [s]
            continue
        current['text_lines'].append(s)

    save()
    
    for i, c in enumerate(chunks):
        c['meta']['chunk_index'] = i
        c['meta']['ingestion_date'] = datetime.now(timezone.utc).isoformat()
    
    print(f'✅ Chunking: {len(chunks)} chunks')
    return chunks

print('✅ Chunking listo')


In [ ]:
# 8) Modelo de embeddings
print('🧠 Cargando modelo de embeddings...')
model = SentenceTransformer('sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2')
print(f'✅ Modelo cargado — dimensión: {model.get_sentence_embedding_dimension()}')

def make_embeddings(texts, batch_size=32):
    embs = []
    for i in range(0, len(texts), batch_size):
        batch = texts[i:i+batch_size]
        arr = model.encode(batch, show_progress_bar=True, convert_to_numpy=True)
        embs.extend([v.tolist() for v in arr])
    return embs

print('✅ Embeddings listos')


In [ ]:
# 9) Subir a Supabase
def upload_chunks(chunks, conn):
    cur = conn.cursor()
    
    records = []
    for i, c in enumerate(chunks):
        text = clean_text(c['text'])
        if not text: continue
        meta = c.get('meta', {})
        chunk_id = f"{meta.get('file', 'doc')}_{meta.get('chunk_index', i)}"
        records.append((chunk_id, text, json.dumps(meta)))
    
    print(f'📊 Generando embeddings para {len(records)} chunks...')
    embeddings = make_embeddings([r[1] for r in records])
    
    print(f'📤 Subiendo a Supabase...')
    insert_sql = f"""
        INSERT INTO {SCHEMA}.{TABLE} (id, vec, text, metadata) 
        VALUES %s 
        ON CONFLICT (id) DO UPDATE SET vec=EXCLUDED.vec, text=EXCLUDED.text, metadata=EXCLUDED.metadata
    """
    
    to_insert = [(rid, '['+','.join(map(str,emb))+']', txt, meta) 
                 for (rid, txt, meta), emb in zip(records, embeddings)]
    
    execute_values(cur, insert_sql, to_insert, template="(%s, %s::vector, %s, %s::jsonb)")
    conn.commit()
    print(f'✅ Subidos {len(to_insert)} chunks')
    cur.close()

print('✅ Función de subida lista')


In [ ]:
# 10) 🚀 EJECUTAR INGESTA DESDE WORD
# ⚠️ IMPORTANTE: Este script BORRA todos los datos anteriores de Supabase

if not os.path.exists(WORD_FILE):
    print(f"❗ No se encuentra: {WORD_FILE}")
    print("   Sube el archivo Word (.docx) antes de ejecutar esta celda.")
    print("\n   📁 Clic en el ícono de carpeta (izquierda) → Subir archivo")
else:
    print('='*50)
    print('🚀 INGESTA DESDE WORD (.docx)')
    print('='*50)
    
    # 1) Extraer texto del Word (¡viene limpio!)
    print('\n📄 Paso 1: Leyendo archivo Word...')
    data = read_word_extract_text(WORD_FILE)
    
    # 2) Generar chunks
    print('\n✂️ Paso 2: Generando chunks por artículos/capítulos...')
    chunks = chunk_hierarchical_legal(data['text'])
    for i, c in enumerate(chunks):
        c['meta']['file'] = WORD_FILE
    
    # 3) Conectar a Supabase
    print('\n🔗 Paso 3: Conectando a Supabase...')
    conn = get_connection()
    
    # 4) BORRAR DATOS ANTERIORES
    print('\n🗑️ Paso 4: BORRANDO datos anteriores de Supabase...')
    cur = conn.cursor()
    cur.execute(f"SELECT count(*) FROM {SCHEMA}.{TABLE}")
    datos_anteriores = cur.fetchone()[0]
    print(f'   Datos anteriores: {datos_anteriores} chunks')
    cur.execute(f"DELETE FROM {SCHEMA}.{TABLE};")
    conn.commit()
    cur.close()
    print('   ✅ Tabla limpiada completamente')
    
    # 5) Subir chunks con embeddings
    print('\n📤 Paso 5: Subiendo chunks con embeddings...')
    upload_chunks(chunks, conn)
    
    # 6) Verificar resultado
    print('\n🔍 Paso 6: Verificando...')
    cur = conn.cursor()
    cur.execute(f"SELECT count(*) FROM {SCHEMA}.{TABLE}")
    total = cur.fetchone()[0]
    cur.close()
    conn.close()
    
    print('\n' + '='*50)
    print(f'✅ INGESTA COMPLETADA EXITOSAMENTE')
    print(f'   📊 Chunks subidos: {total}')
    print(f'   📄 Fuente: Word (.docx) - texto limpio')
    print('='*50)
    print('\n⚠️ Ahora ve a Railway y haz REDEPLOY de tu bot.')


# 11) 🧪 TEST: Verificar el artículo 52
conn = get_connection()
cur = conn.cursor()
cur.execute(f"""
    SELECT LEFT(text, 600) 
    FROM {SCHEMA}.{TABLE} 
    WHERE text ILIKE '%artículo 52%' 
    LIMIT 1
""")
result = cur.fetchone()
cur.close()
conn.close()

if result:
    print('✅ Artículo 52 encontrado:')
    print('-'*50)
    print(result[0])
    print('-'*50)
    print('\n👆 Verifica que el texto esté LIMPIO (sin palabras cortadas)')
else:
    print('⚠️ No se encontró el artículo 52')
